In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
!pip install wandb

In [2]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 9.2 MB/s eta 0:00:00


In [3]:
import wandb
from google.colab import userdata
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import ORPOConfig, ORPOTrainer
from transformers import TextStreamer
import pprint

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.5.1+cu124)
    Python  3.11.8 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
wb_token = "45e7cdcaa9d14a56a493a13603b5bd3059289e78"
wandb.login(key=wb_token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rdidhitpatel2002 (rdidhitpatel2002-bytevitadel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=8192,
    dtype=None,
    load_in_4bit=True
)

==((====))==  Unsloth 2025.2.4: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [10]:
dataset = load_dataset("reciperesearch/dolphin-sft-v0.1-preference")["train"]

README.md:   0%|          | 0.00/490 [00:00<?, ?B/s]

dpo_fixed.jsonl:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def format_prompt(sample):
    instruction = sample["instruction"]
    input       = sample["input"]
    accepted    = sample["accepted"]
    rejected    = sample["rejected"]

    # ORPOTrainer expects prompt/chosen/rejected keys
    # See: https://huggingface.co/docs/trl/main/en/orpo_trainer
    sample["prompt"]   = alpaca_prompt.format(instruction, input, "")
    sample["chosen"]   = accepted + EOS_TOKEN
    sample["rejected"] = rejected + EOS_TOKEN
    return sample
pass

In [12]:
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [13]:
row = dataset[1]
print('INSTRUCTION: ' + '=' * 50)
pprint.pprint(row["prompt"])
print('ACCEPTED: ' + '=' * 50)
pprint.pprint(row["chosen"])
print('REJECTED: ' + '=' * 50)
pprint.pprint(row["rejected"])

INSTRUCTION: ==================================================
('Below is an instruction that describes a task, paired with an input that '
 'provides further context. Write a response that appropriately completes the '
 'request.\n'
 '\n'
 '### Instruction:\n'
 'You are an AI assistant that helps people find information.\n'
 '\n'
 '### Input:\n'
 'Given the rationale, provide a reasonable question and answer. Step-by-step '
 'reasoning process: Xkcd comics are very popular amongst internet users.\n'
 ' The question and answer:\n'
 '\n'
 '### Response:\n')
ACCEPTED: ==================================================
('Question: What makes Xkcd comics popular among internet users?\n'
 '\n'
 'Answer: Xkcd comics are popular among internet users because of their clever '
 'humor, relatable themes, and minimalist art style. They often cover topics '
 'like science, technology, and life experiences, making them appealing to a '
 'broad audience.<|end_of_text|>')
REJECTED: =================

In [14]:
train_dataset = dataset.shuffle(seed=10).select(range(3000))
eval_dataset = dataset.shuffle(seed=10).select(range(1000))

In [15]:
orpo_args = ORPOConfig(
        max_length = 8192,
        max_prompt_length = 8192//2,
        max_completion_length = 8192//2,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        beta = 0.1, # optimization process
        logging_steps = 1, #frequency of logging metrics
        optim = "adamw_8bit",
        lr_scheduler_type = "linear",
        max_steps = 10, # Change to num_train_epochs = 1 for full training runs
        fp16 = not torch.cuda.is_bf16_supported(), # Whether to use 16-bit floating-point precision for training
        bf16 = torch.cuda.is_bf16_supported(), # Whether to use 16-bit bfloat16 precision for training
        output_dir = "outputs",
        report_to="wandb",
    )

In [16]:
orpo_trainer = ORPOTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=orpo_args
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
orpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,8.279300
2,6.560500
3,6.296200
4,5.613900
5,9.106900
6,6.879100
7,7.897500
8,5.018700
9,6.690600
10,5.860700


TrainOutput(global_step=10, training_loss=6.820331716537476, metrics={'train_runtime': 223.2113, 'train_samples_per_second': 0.358, 'train_steps_per_second': 0.045, 'total_flos': 0.0, 'train_loss': 6.820331716537476, 'epoch': 0.02666666666666667})

In [18]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.", # instruction
        "Solve the following algebraic equation: 2x + 5 = 15", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.

### Input:
Solve the following algebraic equation: 2x + 5 = 15

### Response:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task, provide further context. Write a response that describes the task and provides instructions.

### Task:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task, provide further context. Write a response that describes the task and provides instructions.

### Task:
You are an AI assistant. User will you give you a

In [19]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.", # instruction
        "Describe the steps to take when preparing for a job interview.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an AI assistant. User will you give you a task. Your goal is to complete the task as faithfully as you can. While performing the task think step-by-step and justify your steps.

### Input:
Describe the steps to take when preparing for a job interview.

### Response:
You are an AI assistant. User will you give you a task. Your goal is to complete the request.

### Task:
You are an AI assistant. User will you give you a task. Your goal is to complete the request.

### Request:
You are an AI assistant. User will you give you a task. Your goal is to complete the request.

### Task:
You are an AI assistant. User will you give you a task. Your goal is to complete the request.

### Request:
You are an AI assistant. User will you give you a task. Your goal is to complete the request.

### Task:
Y

In [20]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [53]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass


In [83]:
!pip install pyngrok --quiet
!pip install mlflow

import mlflow
from pyngrok import ngrok

# Terminate open tunnels if they exist
ngrok.kill()

# Set your ngrok authtoken
# Get it from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open a ngrok tunnel to the MLflow server
ngrok_tunnel = ngrok.connect

In [90]:
!pip install pyngrok --quiet
!pip install mlflow
!pip install pyngrok --quiet
!pip install mlflow

import mlflow
from pyngrok import ngrok

# Terminate open tunnels if they exist
ngrok.kill()

# Set your ngrok authtoken
# Get it from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = "2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq"
ngrok.set_auth_token(NGROK_AUTH_TOKEN) # ✅ Only pass the token string

# Open a ngrok tunnel to the MLflow server on port 5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
public_url = ngrok_tunnel.public_url # Get the public URL from the tunnel object
print(f"MLflow Tracking UI: {public_url}")
mlflow.set_tracking_uri(public_url)  # Set the MLflow tracking URI to the public URL
# Set or create the experiment
mlflow.set_experiment("Anomaly Detection")

# ... (rest of your code) ...

ERROR:pyngrok.process.ngrok:t=2025-02-09T23:10:52+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: ngrok config add-authtoken 2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: ngrok config add-authtoken 2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [76]:
MLFLOW_TRACKING_URI = "http://127.0.0.1:8080"


In [78]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist


In [80]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

NGROK_AUTH_TOKEN = "2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq"  # ✅ Correct
ngrok.set_auth_token(NGROK_AUTH_TOKEN)  # ✅ Set auth token using pyngrok function
port=8080
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

ERROR:pyngrok.process.ngrok:t=2025-02-09T23:02:50+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: ngrok config add-authtoken 2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-09T23:02:51+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: ngrok config add-authtoken 2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-02-09T23:02:51+0000 lvl=eror msg="t

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok tunnel authtoken.\nYour authtoken: ngrok config add-authtoken 2sMbLQ1R18HpagpdaQEysjFdbUq_4Mf4w2DoBjPeCpMBrTCVq\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.

In [37]:
from google.colab import userdata



ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://c82d-34-169-45-232.ngrok-free.app


In [46]:
import mlflow

mlflow.tracking.set_tracking_uri("http://localhost:5000")  # Set the local tracking URL




In [47]:
# Start MLflow run

with mlflow.start_run():
    # Log hyperparameters
    mlflow.log_params({
        "max_length": orpo_args.max_length,
        "batch_size": orpo_args.per_device_train_batch_size,
        "beta": orpo_args.beta,
        "optimizer": orpo_args.optim,
        "max_steps": orpo_args.max_steps,
        "fp16": orpo_args.fp16,
        "bf16": orpo_args.bf16,
    })

    # Train the model
    orpo_trainer.train()

    # Log training metrics
    for log in orpo_trainer.state.log_history:
        if "loss" in log:
            mlflow.log_metric("train_loss", log["loss"], step=log.get("step"))

    # Log the model and tokenizer
    model.save_pretrained("lora_model")
    tokenizer.save_pretrained("lora_model")
    mlflow.log_artifacts("lora_model", artifact_path="lora_model")

    # Log inference examples
    def log_inference_example(instruction, input_text):
        prompt = alpaca_prompt.format(instruction, input_text, "")
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=128)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        mlflow.log_text(output_text, f"inference_{input_text[:10]}.txt")

    log_inference_example(
        "You are an AI assistant...",
        "Solve the following algebraic equation: 2x + 5 = 15"
    )
    log_inference_example(
        "You are an AI assistant...",
        "Describe the steps to take when preparing for a job interview."
    )

KeyboardInterrupt: 